In [0]:

%sql
drop table if exists employee_src_scd3;
CREATE TABLE if not exists employee_src_scd3 (
    emp_id     INT,
    emp_name   VARCHAR(50),
    dept       VARCHAR(50)
)
USING DELTA;
INSERT INTO employee_src_scd3 VALUES
(1, 'Alice', 'HR'),
(2, 'Bob', 'IT'),
(3, 'Charlie', 'Finance');

drop table if exists employee_dim_scd3;

CREATE TABLE if not EXISTS employee_dim_scd3 (
    emp_id      INT,
    emp_name    VARCHAR(50),
    dept        VARCHAR(50),
    start_date  DATE,
    end_date    DATE,
    is_current  CHAR(1)
)
USING DELTA;
INSERT INTO employee_dim_scd3
SELECT
    emp_id,
    emp_name,
    dept,
    CURRENT_DATE,
    NULL,
    'Y'
FROM employee_src_scd3;

In [0]:

%sql
--SCD2

--Updating the existing record if the value is changed

MERGE INTO employee_dim_scd3 d
USING employee_src_scd3 s
ON d.emp_id = s.emp_id
AND d.is_current = 'Y'

WHEN MATCHED
AND (d.emp_name <> s.emp_name OR d.dept <> s.dept)
THEN UPDATE SET
  d.end_date = CURRENT_DATE - 1,
  d.is_current = 'N'

WHEN NOT MATCHED
THEN INSERT (
  emp_id,
  emp_name,
  dept,
  start_date,
  end_date,
  is_current
)
VALUES (
  s.emp_id,
  s.emp_name,
  s.dept,
  CURRENT_DATE,
  NULL,
  'Y'
);




In [0]:
%sql
drop table if exists employee_src_scd1;
CREATE TABLE if not exists employee_src_scd1 (
    emp_id     INT,
    emp_name   VARCHAR(50),
    dept       VARCHAR(50)
);
INSERT INTO employee_src_scd1 VALUES
(1, 'Alice', 'HR'),
(2, 'Bob', 'IT'),
(3, 'Charlie', 'Finance');

drop table if exists employee_dim_scd1;

CREATE TABLE if not EXISTS employee_dim_scd1 (
    emp_id      INT,
    emp_name    VARCHAR(50),
    dept        VARCHAR(50)
);

--First load
INSERT INTO employee_dim_scd1
SELECT
    emp_id,
    emp_name,
    dept
FROM employee_src_scd1;


In [0]:
%sql
UPDATE employee_src_scd1 set dept='HR' where emp_id=3;
UPDATE employee_src_scd1 set emp_name='John' where emp_id=2

In [0]:
%sql
MERGE INTO employee_dim_scd1 d
USING employee_src_scd1 s
ON d.emp_id = s.emp_id

WHEN MATCHED
AND (d.dept <> s.dept OR d.emp_name <> s.emp_name)
THEN UPDATE SET
  d.emp_name = s.emp_name,
  d.dept = s.dept;



In [0]:
%sql
SHOW GRANTS ON TABLE workspace.default.employee_dim_scd2;

